In [1]:
!sudo apt-get install libmetis-dev
!pip install metis
import metis
import random
import tensorflow as tf
import numpy as np
from scipy import sparse
import scipy.io as sio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmetis-dev is already the newest version (5.1.0.dfsg-5).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [2]:
# hyperparameters
hidden = 512 # number of hidden units in the encoder layer
latent = 256 # dimension of the latent variables
learning_rate = 0.01
epochs = 200
nparts = 1500 # number of partitions
batch_size = 20 # number of clusters per batch

In [3]:
filename = '/content/drive/MyDrive/GRAPH DATA/reddit.mat' # dataset

In [4]:
mat_dict = sio.loadmat(filename)
A = mat_dict['A'].ceil()
X = mat_dict['X']
Y = mat_dict['Y']
train_mask = mat_dict['train_mask'].squeeze().astype(bool)
val_mask = mat_dict['val_mask'].squeeze().astype(bool)
test_mask = mat_dict['test_mask'].squeeze().astype(bool)

In [5]:
def cluster_graph(A, nparts):
  if nparts == 1:
    edge_cuts, parts = 0, [0, ] * A.shape[0]
  else:
    edge_cuts, parts = metis.part_graph([neighbors for neighbors in A.tolil().rows], nparts=nparts)
  print('Number of edge cuts: %d.' % edge_cuts)
  cluster_dict = {}
  for index, part in enumerate(parts):
    if part not in cluster_dict:
      cluster_dict[part] = []
    cluster_dict[part].append(index)
  return cluster_dict

# the clustering algorithm (METIS)
cluster_dict = cluster_graph(A, nparts)

Number of edge cuts: 9609639.


In [6]:
def preprocess_support(A):
  N = A.shape[1]
  A_I = A + sparse.eye(N, dtype='float32')
  D_I = sparse.csr_matrix(A_I.sum(axis=1))
  norm = D_I.power(-0.5)
  return A_I.multiply(norm).T.multiply(norm)

def toTensorSparse(S):
  return tf.constant(S.todense())

def toTensor(T):
  return tf.constant(T)

In [7]:
# layer classes

class bilinear_layer:

  def __init__(self, indim, outdim):
    pass

  def __call__(self, tensor):
    return tf.linalg.matmul(tensor, tf.transpose(tensor))

# unused
class FC_layer:

  def __init__(self, indim, outdim):
    initial_value = tf.initializers.he_normal()((indim, outdim,))
    self.weight = tf.Variable(initial_value=initial_value, trainable=True)

  def __call__(self, tensor):
    return tf.linalg.matmul(tensor, self.weight)

class GC_layer:

  def __init__(self, indim, outdim):
    initial_value = tf.initializers.he_normal()((indim, outdim,))
    self.weight = tf.Variable(initial_value=initial_value, trainable=True)

  def __call__(self, tensor, support, embed=False):
    if embed: # numpy pipeline
      return support.dot(tensor.numpy().dot(self.weight.numpy()))
    else: # tensorflow pipeline
      return tf.linalg.matmul(support, tf.linalg.matmul(tensor, self.weight))


In [8]:
# our model class (for the paper "Scalable Graph Variational Autoencoders")

class Model:

  def __init__(self, size_tuple, optimizer, nonlinear):
    self.sources = [] # variables to optimize
    self.build(size_tuple) # builds the model by stacking layers on each other
    self.optimizer = optimizer
    self.nonlinear = nonlinear
    self.Z_mean = None # mean embedding layer
    self.Z_var = None # variance embedding layer
    self.noise = None # the noise sample
    self.sample = None # self.Z_mean + self.Z_var * self.noise
    self.A_gamma = None # the reconstructions
  
  def build(self, size_tuple):
    X_dim, hidden, latent = size_tuple
    self.enc_layer = GC_layer(X_dim, hidden)
    self.enc_mean_layer = GC_layer(hidden, latent)
    self.enc_var_layer = GC_layer(hidden, latent)
    self.A_dec_gamma_layer = bilinear_layer(latent, latent)
    # filling the source array with weights
    layers = [self.enc_layer, self.enc_mean_layer, self.enc_var_layer]
    for layer in layers:
      self.sources.append(layer.weight)
  
  # forward propagation in the encoder
  def encode(self, X, S):
    enc = self.nonlinear(self.enc_layer(X, S))
    enc_mean = self.enc_mean_layer(enc, S)
    enc_var = tf.math.exp(self.enc_var_layer(enc, S))
    return enc_mean, enc_var

  # returns only the node embeddings
  def embed(self, X, S):
    enc = self.nonlinear(self.enc_layer(X, S, embed=True))
    enc_mean = self.enc_mean_layer(enc, S, embed=True)
    return enc_mean

  # forward propagation in the decoder
  def decode(self, sample):
    A_dec_gamma = self.A_dec_gamma_layer(sample)
    return A_dec_gamma

  def predict(self, X, S):
    self.Z_mean, self.Z_var = self.encode(X, S)
    self.noise = tf.random.normal(self.Z_var.shape)
    self.sample = self.Z_mean + self.Z_var * self.noise # reparameterization trick
    self.A_gamma = self.decode(self.sample)

  def train(self, X, A, cluster_dict, batch_size, epochs):
    for epoch in range(epochs):
      # only a subgraph is used in the training process
      samples = random.sample(cluster_dict.keys(), batch_size)
      nodes = sum([cluster_dict[sample] for sample in samples], [])
      S_batch = toTensorSparse(preprocess_support(A[nodes].T[nodes]))
      A_batch = toTensor(A.T[nodes].T[nodes].todense())
      X_batch = tf.math.l2_normalize(toTensor(X[nodes]), axis=1)
      # optimization
      with tf.GradientTape() as tape:
        self.predict(X_batch, S_batch)
        losses = self.loss(A_batch, X_batch)
        loss_ = tf.reduce_sum(losses)
      print(epoch, [loss.numpy() for loss in losses], loss_.numpy())
      grads = tape.gradient(loss_, self.sources)
      self.optimizer.apply_gradients(zip(grads, self.sources))

  # Kullback–Leibler divergence
  def KL_Divergence(self):
    loss = 0.5 * tf.reduce_mean(self.Z_mean**2.0 + self.Z_var**2.0 - 2.0 * tf.math.log(self.Z_var) - 1.0)
    return loss

  # reconstruction loss
  def re_A_loss(self, A):
    density = tf.reduce_sum(A) / tf.size(A, out_type=tf.float32)
    pos_weight = (1.0 - density) / density
    loss = -0.5 * tf.reduce_mean(1.0 / (1.0 - density) * tf.nn.weighted_cross_entropy_with_logits(labels=A, logits=self.A_gamma, pos_weight=pos_weight))
    return -loss

  # list of all loss functions
  def loss(self, A, X):
    return self.KL_Divergence(), self.re_A_loss(A)


In [9]:
size_tuple = (X.shape[1], hidden, latent)
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
nonlinear = tf.nn.relu

model = Model(size_tuple, optimizer, nonlinear)

print('Training...')
model.train(X, A, cluster_dict, batch_size, epochs)


Training...
0 [0.0047069215, 6.589947] 6.594654
1 [0.021607213, 5.0971265] 5.118734
2 [0.07974266, 3.5981739] 3.6779165
3 [0.23128651, 2.096803] 2.3280895
4 [0.4854414, 1.4227942] 1.9082355
5 [0.8196651, 1.0161958] 1.8358608
6 [1.1136945, 1.6832622] 2.7969568
7 [1.191991, 0.92631495] 2.118306
8 [1.2690568, 1.5022756] 2.7713323
9 [1.264591, 0.9664697] 2.2310607
10 [1.148089, 1.0598824] 2.2079716
11 [1.0869949, 0.83456075] 1.9215556
12 [0.9579296, 0.8477148] 1.8056444
13 [0.7978228, 0.88878906] 1.6866119
14 [0.750253, 0.9027708] 1.6530238
15 [0.59232956, 0.991692] 1.5840216
16 [0.50038934, 1.1714028] 1.6717921
17 [0.41803837, 1.3114457] 1.7294841
18 [0.3988082, 1.4030156] 1.8018239
19 [0.41451365, 1.3779553] 1.792469
20 [0.4267816, 1.3047972] 1.7315788
21 [0.46418837, 1.2414572] 1.7056456
22 [0.52551746, 1.1128404] 1.6383579
23 [0.6161648, 1.0142229] 1.6303877
24 [0.6882699, 0.92302805] 1.611298
25 [0.69762677, 0.90551037] 1.6031371
26 [0.7391903, 0.88411605] 1.6233063
27 [0.78177667, 0.

In [10]:
S = preprocess_support(A)
X = tf.math.l2_normalize(toTensor(X), axis=1)
embs = model.embed(X, S) # node embeddings

In [11]:
# node clustering using the KMeans algorithm
from sklearn.cluster import KMeans
y_pred = KMeans(n_clusters=Y.shape[1]).fit(embs).predict(embs)
y_true = np.argmax(Y, axis=1)

In [12]:
# result
from sklearn.metrics import adjusted_mutual_info_score
print(adjusted_mutual_info_score(y_true, y_pred))

0.44621126475467937
